In [40]:
from bokeh.plotting import figure, show
from bokeh.io import output_notebook

output_notebook()

import json

Loading BokehJS ...

In [41]:
userScores_simple = json.load(open("userScores_simple.json"))
userScores = json.load(open("userScores.json"))

In [42]:
insiders = ["ABB0427", "HFC0492", "KTW0365"]

def getRocPoints(userScores):
    rocPoints = []

    users = userScores.keys()

    for T in range(0,10000,10):
        anomUsers = set()
        for user in users:
            for weekScore in userScores[user]:
                if weekScore > T :
                    anomUsers.add(user)

        if len(anomUsers) == 0:
            break

        insiders_caught = 0
        for insider in insiders:
            if insider in anomUsers:
                insiders_caught += 1

        true_positive = insiders_caught/len(insiders)
        false_positive = (len(anomUsers)-insiders_caught)/len(users)
        rocPoints.append( (false_positive, true_positive) )
        
    # why is everything an iterator...
    return list(reversed(rocPoints))

In [43]:
def getAUC(rocPoints):
    auc = 0
    prev = (0,0)
    for x in rocPoints:
        auc += (x[0]-prev[0])*(prev[1]+x[1])/2
        prev = x
    return auc

In [44]:
p = figure(x_range=(0,1), y_range=(0,1.001))
p.title = "ROC Curve for Dataset 4.1"
p.xaxis.axis_label = "α"
p.yaxis.axis_label = "1-ß"

def drawRoc(rocPoints, p, c, name):

    xp = list(map(lambda x : x[0], rocPoints))
    yp = list(map(lambda x : x[1], rocPoints))

    p.line(xp,yp, line_color=c, legend=name)
    # p.scatter(xp,yp, marker="x", size=10, color=c)

p.line([0,1],[0,1],color="black")

simpleRoc = getRocPoints(userScores)
simplestRoc = getRocPoints(userScores_simple)

drawRoc(simpleRoc, p, "blue", "Simple")
drawRoc(simplestRoc, p, "red", "Simplest")

print("  Simple features AUC: ", getAUC(simpleRoc))
print("Simplest features AUC: ", getAUC(simplestRoc))

show(p)

  Simple features AUC:  0.9195
Simplest features AUC:  0.9083333333333333
